<a href="https://colab.research.google.com/github/MOOwuttichai/BSC_DPDM2023/blob/main/%E0%B8%BABERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [122]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

In [123]:
df = pd.read_csv("/content/Data_model.csv")
texts = df['comments'].tolist()

In [124]:
# prompt: ช่วยเเก้ไข Imbalance ของข้อมูล df โดยที่ x คือคอลัม comments, y คือคอลัม label โดยใช้ imblearn

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Separate features (X) and target (y)
X = df['comments'].values.reshape(-1, 1)  # Reshape for imblearn
y = df['label']

# Choose either oversampling or undersampling
# Oversampling:
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X, y)

# # Undersampling:
# undersampler = RandomUnderSampler(random_state=42)
# X_resampled, y_resampled = undersampler.fit_resample(X, y)

# Create a new DataFrame with the resampled data
df_resampled = pd.DataFrame({'comments': X_resampled.flatten(), 'label': y_resampled})

# Now use df_resampled instead of df for training
texts = df_resampled['comments'].tolist()
labels = []
for sentiment in df_resampled['label']:
  if sentiment == "เล่าประสบการณ์ (tell experience)":
    labels.append(1) #เล่าประสบการณ์ (tell experience)
  elif sentiment == "คำถาม (Question)":
    labels.append(2) #ไม่มีประโยชน์/ไม่สำคัญ (useless/unimportant)
  else:
    labels.append(0)
print(len(texts))
print(set(labels))

6771
{0, 1, 2}


In [125]:
# prompt: ช่วยเเปลงข้อมูลใน list จาก str เป็น int

# Assuming 'labels' is a list of strings representing integers


In [126]:
set(labels)

{0, 1, 2}

In [127]:
class TextClassificationDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
  def __len__(self):
      return len(self.texts)
  def __getitem__(self, idx):
      text = self.texts[idx]
      label = self.labels[idx]
      encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
      return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

In [128]:
class BERTClassifier(nn.Module):
  def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)
  def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

In [129]:
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

In [130]:
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

In [131]:
def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
    if preds.item() == 1:
      L = "เล่าประสบการณ์ (tell experience)"
    elif preds.item() == 2:
      L = "คำถาม (Question)"
    else:
      L = "ไม่มีประโยชน์/ไม่สำคัญ (useless/unimportant)"
    return L

In [132]:
# Set up parameters
bert_model_name = 'airesearch/wangchanberta-base-att-spm-uncased'
num_classes = 3
max_length = 200
batch_size = 16
num_epochs = 4
learning_rate = 2e-5

In [133]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.3, random_state=42)

In [134]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [135]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)

You are using a model of type camembert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertModel were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 

In [136]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [137]:
for epoch in range(num_epochs):
  print(f"Epoch {epoch + 1}/{num_epochs}")
  train(model, train_dataloader, optimizer, scheduler, device)
  accuracy, report = evaluate(model, val_dataloader, device)
  print(f"Validation Accuracy: {accuracy:.4f}")
  print(report)

Epoch 1/4
Validation Accuracy: 0.8996
              precision    recall  f1-score   support

           0       1.00      0.87      0.93       689
           1       0.83      0.96      0.89       676
           2       0.89      0.87      0.88       667

    accuracy                           0.90      2032
   macro avg       0.91      0.90      0.90      2032
weighted avg       0.91      0.90      0.90      2032

Epoch 2/4
Validation Accuracy: 0.9665
              precision    recall  f1-score   support

           0       0.99      0.91      0.95       689
           1       0.91      0.99      0.95       676
           2       1.00      1.00      1.00       667

    accuracy                           0.97      2032
   macro avg       0.97      0.97      0.97      2032
weighted avg       0.97      0.97      0.97      2032

Epoch 3/4
Validation Accuracy: 0.9621
              precision    recall  f1-score   support

           0       0.99      0.89      0.94       689
           1   

In [140]:
torch.save(model.state_dict(), "bert_classifier.pth")

In [139]:
test_text = "พึ่งไปนอนรพ.อยู่สามวัน เริ่มมีอาการชีพจรเบา 40-50/นาที เมื่อเมย.นี้ ไม่มีอาการอื่น เมื่อวันที่9 นี้ตื่นมามีอาการเหมือนที่คุณหมอ ว่านี้ ทำEkg,อุลต้าซาว.ออกรพ.แพทย์เพิ่มยาลดความดัน,ยาไขมัน นัดEco แต่ทำไม่ได้ ความดันขึ้นสูง ก็ยังไม่รู้เลยค่ะว่าเป็นอย่างไร"
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print("พึ่งไปนอนรพ.อยู่สามวัน เริ่มมีอาการชีพจรเบา 40-50/นาที เมื่อเมย.นี้ ไม่มีอาการอื่น เมื่อวันที่9 นี้ตื่นมามีอาการเหมือนที่คุณหมอ ว่านี้ ทำEkg,อุลต้าซาว.ออกรพ.แพทย์เพิ่มยาลดความดัน,ยาไขมัน นัดEco แต่ทำไม่ได้ ความดันขึ้นสูง ก็ยังไม่รู้เลยค่ะว่าเป็นอย่างไร")
print(f"Predicted sentiment: {sentiment}")

พึ่งไปนอนรพ.อยู่สามวัน เริ่มมีอาการชีพจรเบา 40-50/นาที เมื่อเมย.นี้ ไม่มีอาการอื่น เมื่อวันที่9 นี้ตื่นมามีอาการเหมือนที่คุณหมอ ว่านี้ ทำEkg,อุลต้าซาว.ออกรพ.แพทย์เพิ่มยาลดความดัน,ยาไขมัน นัดEco แต่ทำไม่ได้ ความดันขึ้นสูง ก็ยังไม่รู้เลยค่ะว่าเป็นอย่างไร
Predicted sentiment: เล่าประสบการณ์ (tell experience)
